# Apprentissage par renforcement

In [13]:
import numpy as np
import pandas as pd
import time

# N_ETATS : Le nombre d'états possibles avant d'atteindre la cible.
# ACTIONS : Les actions possibles de l'agent ('gauche' ou 'droite').
# EPSILON : La probabilité de choisir une action au hasard plutôt que la meilleure action connue, pour explorer l'environnement.
# ALPHA : Le taux d'apprentissage, qui détermine à quel point les nouvelles informations remplacent les anciennes.
# GAMMA : Le facteur de remise, qui détermine l'importance des récompenses futures.
# MAX_CYCLES : Le nombre maximum de cycles d'entraînement.
# TPS_MOUV : Le temps entre les mouvements, pour la simulation.

# A---CIBLE
# Le nombre de positions possibles avant d'atteindre la cible avec le moins de déplacements possible
N_ETATS = 5
# Les actions réalisables par l'agent
ACTIONS = ['gauche', 'droite']
# La valeur qui permet d'indiquer si une action de l'agent est aléatoire ou encore basée sur la qualité
EPSILON = 0.9
# Le taux d'apprentissage qui varie de 0 (l'agent n'a rien appris) à 1 (l'agent ne tient compte que de la dernière
#information apprise)
ALPHA = 0.1
# Le coefficient de réduction
GAMMA = 0.7
# Le nombre de cycles
MAX_CYCLES = 5
# La vitesse des mouvements
TPS_MOUV = 0.1

# Construction d'une Q table
def Construire_q_table(n_etats, actions):
    """
    Construit une Q-table initialisée à zéro pour un nombre donné d'états et d'actions.

    :param n_etats: Nombre d'états dans l'environnement.
    :param actions: Liste des actions possibles.
    :return: DataFrame pandas représentant la Q-table.
    """
    table = pd.DataFrame(np.zeros((n_etats, len(actions))),columns=actions)
    print(f"\nQ table :\n{table} \n")
    return table

# Choix de l'action en fonction de l'état et de la Q table
def choix_action(etat, q_table):
    """
    Sélectionne une action pour un état donné en utilisant la politique epsilon-greedy.

    :param etat: L'état actuel de l'agent.
    :param q_table: La Q-table utilisée pour choisir l'action.
    :return: L'action choisie.
    """
    etat_actions = q_table.iloc[etat, :]
    print(f"\netat_actions :\n{etat_actions} \n")
    if (np.random.uniform() > EPSILON) or ((etat_actions ==0).all()):
        nom_action = np.random.choice(ACTIONS)
    else:
        nom_action = etat_actions.idxmax()
    print(f"\nnom_action :\n{nom_action} \n")
    return nom_action

# Fonction qui indique le nouvel état et la récompense en
#fonction de l'état et de l'action
def nouv_etat_r(E, A):
    """
    Détermine le nouvel état et la récompense basés sur l'état actuel et l'action prise.

    :param E: L'état actuel.
    :param A: L'action effectuée.
    :return: Le nouvel état et la récompense associée.
    """
    if A == 'droite':
        if E == N_ETATS - 2:
            E_ = 'cible'
            R = 1
        else:
            E_ = E + 1
            R = 0
    else:
        R = 0
        if E == 0:
            E_ = E
        else:
            E_ = E - 1
    print(f"\nE_ :\n{E_} ,R  {R} \n")
    return E_, R

# Fonction qui met à jour l'environnement
def maj_env(S, cycle, c_etape):
    """
    Met à jour l'environnement de simulation en affichant l'état actuel et le cycle.

    :param S: L'état actuel ou 'cible' si atteint.
    :param cycle: Le numéro du cycle actuel.
    :param c_etape: Le compteur d'étapes dans le cycle actuel.
    """
    env_list = ['-']*(N_ETATS-1) + ['Cible']
    # print(f"\nenv_list :\n{env_list} \n")
    if S == 'cible':
        interaction = 'Cycle %s: nombre_pas = %s' % (cycle+1,c_etape)
        print('\r{}'.format(interaction), end='')
        time.sleep(1)
        print('\r', end='')
    else:
        env_list[S] = 'A'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(TPS_MOUV)

def apprentissage_renforcement():
    """
    Cette fonction implémente l'algorithme d'apprentissage par renforcement.
    Elle construit une table Q et effectue un certain nombre de cycles d'apprentissage.
    Chaque cycle consiste en une série d'étapes jusqu'à atteindre l'état 'cible'.
    À chaque étape, une action est choisie et l'état est mis à jour en fonction de cette action.
    La table Q est mise à jour à chaque étape en utilisant l'équation de Bellman.
    """

    # Construire la table Q initiale
    q_table = Construire_q_table(N_ETATS, ACTIONS)
    print(f"\nq_table :\n{q_table} \n")
    # Boucle sur le nombre maximal de cycles
    for cycle in range(MAX_CYCLES):
        c_etape = 0  # Initialiser le compteur d'étapes
        E = 0  # Initialiser l'état
        fin_cycle = False  # Indicateur de fin de cycle

        # Mettre à jour l'environnement
        maj_env(E, cycle, c_etape)

        # Boucle jusqu'à la fin du cycle
        while not fin_cycle:
            # Choisir une action en fonction de l'état et de la table Q
            A = choix_action(E, q_table)
            # Obtenir le nouvel état et la récompense en fonction de l'état actuel et de l'action choisie
            E_, R = nouv_etat_r(E, A)
            # Prédire la valeur Q pour l'état actuel et l'action choisie
            q_pred = q_table.loc[E, A]
            # Si le nouvel état n'est pas l'état 'cible'
            if E_ != 'cible':
                # Calculer la valeur cible Q en utilisant l'équation de Bellman
                q_cible = R + GAMMA * q_table.iloc[E_, :].max()
            else:
                # Si le nouvel état est l'état 'cible', la valeur cible Q est simplement la récompense
                q_cible = R
                # Indiquer la fin du cycle
                fin_cycle = True
            # Mettre à jour la valeur Q pour l'état actuel et l'action choisie
            q_table.loc[E, A] += ALPHA * (q_cible - q_pred)
            # Mettre à jour l'état
            E = E_
            # Mettre à jour l'environnement
            maj_env(E, cycle, c_etape+1)
            # Incrémenter le compteur d'étapes
            c_etape += 1

    # Retourner la table Q après l'apprentissage
    return q_table


In [14]:
# Lancement de l'algorithme
q_table = apprentissage_renforcement()
print('\r Q-table :\n', q_table)



Q table :
   gauche  droite
0     0.0     0.0
1     0.0     0.0
2     0.0     0.0
3     0.0     0.0
4     0.0     0.0 


q_table :
   gauche  droite
0     0.0     0.0
1     0.0     0.0
2     0.0     0.0
3     0.0     0.0
4     0.0     0.0 

A---Cible
etat_actions :
gauche    0.0
droite    0.0
Name: 0, dtype: float64 


nom_action :
gauche 


E_ :
0 ,R  0 

A---Cible
etat_actions :
gauche    0.0
droite    0.0
Name: 0, dtype: float64 


nom_action :
droite 


E_ :
1 ,R  0 

-A--Cible
etat_actions :
gauche    0.0
droite    0.0
Name: 1, dtype: float64 


nom_action :
droite 


E_ :
2 ,R  0 

--A-Cible
etat_actions :
gauche    0.0
droite    0.0
Name: 2, dtype: float64 


nom_action :
gauche 


E_ :
1 ,R  0 

-A--Cible
etat_actions :
gauche    0.0
droite    0.0
Name: 1, dtype: float64 


nom_action :
gauche 


E_ :
0 ,R  0 

A---Cible
etat_actions :
gauche    0.0
droite    0.0
Name: 0, dtype: float64 


nom_action :
droite 


E_ :
1 ,R  0 

-A--Cible
etat_actions :
gauche    0.0
droite    0

In [10]:
for _ in range(10):
    q_table = apprentissage_renforcement()
    print('\r Q-table :\n', q_table)


 Q-table :ombre_pas = 43
    gauche    droite
0     0.0  0.000158
1     0.0  0.004194
2     0.0  0.057022
3     0.0  0.409510
4     0.0  0.000000
 Q-table :ombre_pas = 40
      gauche    droite
0  0.000000  0.000213
1  0.000000  0.004552
2  0.000034  0.057022
3  0.000000  0.409510
4  0.000000  0.000000
 Q-table :ombre_pas = 41
      gauche    droite
0  0.000000  0.000266
1  0.000005  0.005623
2  0.000034  0.070800
3  0.002563  0.409510
4  0.000000  0.000000
 Q-table :ombre_pas = 43
    gauche    droite
0     0.0  0.000158
1     0.0  0.004194
2     0.0  0.057022
3     0.0  0.409510
4     0.0  0.000000
 Q-table :ombre_pas = 42
    gauche    droite
0     0.0  0.000158
1     0.0  0.004194
2     0.0  0.057022
3     0.0  0.409510
4     0.0  0.000000
 Q-table :ombre_pas = 40
    gauche    droite
0     0.0  0.000158
1     0.0  0.004194
2     0.0  0.057022
3     0.0  0.409510
4     0.0  0.000000
 Q-table :ombre_pas = 46
    gauche    droite
0     0.0  0.000158
1     0.0  0.004194
2     0.0  0.0